# Compute PFN efficiencies

In [1]:
import numpy as np
import h5py
import yaml
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as keras

2023-10-28 19:08:04.053922: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
task_name = "axion1"

with open("../config.yaml") as fin:
    data = yaml.safe_load(fin)
    model_dir = data['model_dir']
    data_dir = data["data_dir"]
    cloud_dir = f"{data['data_dir']}/processed/{task_name}_cloud.npy"

In [3]:
# ~10 seconds
pfn = keras.models.load_model(f"{model_dir}/{task_name}_pfn")

In [13]:
# Grab data (~15 seconds?)
all_jets = []
for particle in ["pi0", "gamma", task_name]:
    print(f"Loading clouds for {particle}...")
    all_jets.append(np.load(f"{data_dir}/processed/{particle}_cloud.npy"))

Loading clouds for pi0...
Loading clouds for gamma...
Loading clouds for axion1...


In [14]:
def get_GeV(particle):
    """
    Get an array of all the energies.
    """
    file = h5py.File(f"{data_dir}/h5/{particle}_40-250GeV_100k.h5")
    return np.array(np.squeeze(file["energy"]))

## Compute all the info we need

In [15]:
E = [get_GeV("pi0"), get_GeV("gamma"), get_GeV(task_name)]

In [16]:
Y_pred = [
    np.argmax(pfn.predict(jets, batch_size=500), axis=1)
    for jets in all_jets
]

200/200 [==============================] - 13s 65ms/step


In [17]:
N = 100000
Y_true = [0]*N + [1]*N + [2]*N

correct = np.sum(np.concatenate(Y_pred) == np.array(Y_true))
print(f"Overall accuracy: {(correct / len(Y_true)):.3f}")

Overall accuracy: 0.996


In [18]:
def compute_efficiencies(particle):
    """
    Particle can be either 0, 1, or 2.
    """
    cutoffs = np.linspace(40, 250, 11)
    
    # EnergyRangeLow, EnergyRangeUp, Eff, EffErrLow, EffErrUp
    table = []
    for i in range(len(cutoffs) - 1):
        low, up = cutoffs[i], cutoffs[i + 1]
        mask = (E[particle] >= low) & (E[particle] < up)
        
        # Background
        eff = round(np.sum(Y_pred[particle][mask] == 2) / np.sum(mask), 5)
        
        # What should errors actually be?
        table.append([low, up, eff, 0.001, 0.001])
    
    return table

In [19]:
os.makedirs(f"./PFN_results/{task_name}_1GeV", exist_ok=True)

tables = {
    "eff_pi0.txt": compute_efficiencies(0),
    "eff_gamma.txt": compute_efficiencies(1),
    f"eff_{task_name}.txt": compute_efficiencies(2)    
}
for path, table in tables.items():
    with open(f"./PFN_results/{task_name}_1GeV/{path}", "w") as fout:
        fout.write("EnergyRangeLow, EnergyRangeUp, Eff, EffErrLow, EffErrUp\n")
        fout.write("\n".join([", ".join(map(str, row)) for row in table]))
        fout.write("\n")